# Basketball-reference Web Scraping Merge Player Data and Salary Data

In [1]:
# special IPython command to prepare the notebook for matplotlib
from collections import defaultdict

import IPython
import os
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re
import string
from pattern import web

print 'IPython version:', IPython.__version__
print 'pandas version:', pd.__version__
print 'requests version:', requests.__version__
print 're version:', re.__version__
#print 'current path:',os.getcwd()

IPython version: 3.1.0
pandas version: 0.15.2
requests version: 2.7.0
re version: 2.2.1


###Team Mapping

In [2]:
team_list = ["Atlanta Hawks","Boston Celtics","Brooklyn Nets","Charlotte Bobcats","Chicago Bulls","Charlotte Hornets","Cleveland Cavaliers","Dallas Mavericks","Denver Nuggets","Detroit Pistons"
             ,"Golden State Warriors","Houston Rockets","Indiana Pacers","Los Angeles Clippers","Los Angeles Lakers","Memphis Grizzlies","Miami Heat","Milwaukee Bucks","Minnesota Timberwolves","New Jersey Nets"
             ,"New Orleans Hornets","New Orleans/Oklahoma City Hornets","New Orleans Pelicans","New York Knicks","Oklahoma City Thunder","Orlando Magic","Philadelphia 76ers","Phoenix Suns","Portland Trail Blazers","Sacramento Kings"
             ,"San Antonio Spurs","Seattle SuperSonics","Toronto Raptors","Utah Jazz","Washington Wizards"]

alias_list = ["ATL","BOS","BRK","CHA","CHI","CHO","CLE","DAL","DEN","DET"
              ,"GSW","HOU","IND","LAC","LAL","MEM","MIA","MIL","MIN","NJN"
              ,"NOH","NOK","NOP","NYK","OKC","ORL","PHI","PHO","POR","SAC"
              ,"SAS","SEA","TOR","UTA","WAS"]

## Read Player list

In [3]:
df_player = pd.read_csv('2014-15 NBA player_list.csv', header=0)

df_player.info()
#df_player.describe()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 562 entries, 0 to 561
Data columns (total 9 columns):
Player        562 non-null object
From          562 non-null int64
To            562 non-null int64
Pos           562 non-null object
Ht            562 non-null object
Wt            562 non-null int64
Birth Date    562 non-null object
College       461 non-null object
link          562 non-null object
dtypes: int64(3), object(6)
memory usage: 43.9+ KB


#Fetch player information page

In [4]:
player_name = df_player['Player'][475]
player_information_url = df_player['link'][475]

print player_name
print player_information_url

#player_website_html = requests.get(player_information_url.values[0]).text
#print player_website_html.decode

Amar'e Stoudemire
http://www.basketball-reference.com/players/s/stoudam01.html


### Player page table

In [5]:
def get_player_html_tables(html):
    """Parse html and return html tables of basketball-reference player data."""

    dom = web.Element(html)
    
    #### 1. step: get all tables
    #tbls = dom('table')
    #### 2. step: get all tables we care about
    tbls = dom.by_class('sortable')
    return tbls

def player_table_type(tbl):
    ### Extract the player_table type
    return tbl.id

def player_information(player_information_url):
    player_website_html = requests.get(player_information_url).text
    #print player_website_html.decode
    player_information_tables = get_player_html_tables(player_website_html)
    
    ### player_table to dict ####
    player_table_by_type = {}
    
    for tbl in player_information_tables:
        typ = player_table_type(tbl)
        if typ not in player_table_by_type:
            player_table_by_type[typ] = list()  # equivalent to []
        player_table_by_type[typ].append(tbl)

    # group the player_tables by type
    player_tables_by_type = defaultdict(list)
    
    # defaultdicts have a default value that is inserted when a new key is accessed
    for tbl in player_information_tables:
        player_tables_by_type[player_table_type(tbl)].append(tbl)
    #print player_tables_by_type
    
    # return player_information_tables,player_tables_by_type
    return (player_information_tables,player_tables_by_type)

### Function call ###
(player_information_tables,player_tables_by_type) = player_information(player_information_url)

print "table length: %d" %len(player_information_tables)
for t in player_information_tables:
    print t.attributes

print "=" *80
    
print player_tables_by_type

table length: 18
{u'class': u'sortable  row_summable stats_table', u'id': u'totals'}
{u'class': u'sortable  row_summable stats_table', u'id': u'per_game'}
{u'class': u'sortable  row_summable stats_table', u'id': u'per_minute'}
{u'class': u'sortable  row_summable stats_table', u'id': u'per_poss'}
{u'class': u'sortable  row_summable stats_table', u'id': u'advanced'}
{u'class': u'sortable  row_summable stats_table', u'id': u'shooting'}
{u'class': u'sortable  row_summable stats_table', u'id': u'advanced_pbp'}
{u'class': u'sortable  row_summable stats_table', u'id': u'playoffs_totals'}
{u'class': u'sortable  row_summable stats_table', u'id': u'playoffs_per_game'}
{u'class': u'sortable  row_summable stats_table', u'id': u'playoffs_per_minute'}
{u'class': u'sortable  row_summable stats_table', u'id': u'playoffs_per_poss'}
{u'class': u'sortable  row_summable stats_table', u'id': u'playoffs_advanced'}
{u'class': u'sortable  row_summable stats_table', u'id': u'playoffs_shooting'}
{u'class': u'so

### Salary

In [6]:
def get_players_salaries(tables):
    """Extract Active players data for players from all tables and store it in dictionary."""
    salaries_dict={}
    salaries_list=[] ### (Season,Team,Lg,Salary)
    
    # 1. step: try to extract data for a single table
    for tbl in tables:
        headers = tbl('tr')
        #print headers
        first_header = headers[0]
        #print first_header
        th_s = first_header('th')
        #print th_s    
        title = [val.content for val in th_s ]
        #print title
                
        # 2. step: iterate over all tables, extract headings and actual data and combine data into single dict
        rows = tbl('tr')[1:-1]
        for row in rows:
            
            tds = row('td')
            
            ### if row content is null , set default string ###
            f_content = (lambda x: "" if len(x) == 0 else x.content)
            ### if row a_herf is null , set default string ###
            f_ahref = (lambda x: "" if len(x) == 0 else x('a')[0].content)
            
            Season    = f_content(tds[0]) ; #print Season
            Team      = f_ahref(tds[1])   ; #print Team
            Tm        = alias_list[team_list.index(Team)] #print Tm
            Lg        = f_ahref(tds[2]) ; #print Lg
            Salary    = f_content(tds[3]) ; #print Salary

            salaries_list=[Season,Team,Tm,Lg,Salary]
            salaries_dict[Season+Tm]=salaries_list

    return salaries_dict


### Fucnction call ###
salaries_dict=get_players_salaries(player_tables_by_type[u'salaries'])

if len(salaries_dict) > 0:
    #print salaries_dict
    df_salaries = pd.DataFrame(salaries_dict.values())
    df_salaries['name'] = player_name
    df_salaries.columns=['Season','Team','Tm','Lg','Salary','Name']    
    print df_salaries.sort_index(by='Season')

     Season              Team   Tm   Lg       Salary               Name
7   2002-03      Phoenix Suns  PHO  NBA   $1,767,120  Amar'e Stoudemire
9   2003-04      Phoenix Suns  PHO  NBA   $1,899,720  Amar'e Stoudemire
13  2004-05      Phoenix Suns  PHO  NBA   $2,032,200  Amar'e Stoudemire
4   2005-06      Phoenix Suns  PHO  NBA   $2,589,023  Amar'e Stoudemire
11  2006-07      Phoenix Suns  PHO  NBA  $12,455,000  Amar'e Stoudemire
5   2007-08      Phoenix Suns  PHO  NBA  $13,762,775  Amar'e Stoudemire
10  2008-09      Phoenix Suns  PHO  NBA  $15,070,550  Amar'e Stoudemire
12  2009-10      Phoenix Suns  PHO  NBA  $16,378,325  Amar'e Stoudemire
0   2010-11   New York Knicks  NYK  NBA  $16,486,611  Amar'e Stoudemire
6   2011-12   New York Knicks  NYK  NBA  $18,217,705  Amar'e Stoudemire
2   2012-13   New York Knicks  NYK  NBA  $19,948,799  Amar'e Stoudemire
3   2013-14   New York Knicks  NYK  NBA  $21,679,893  Amar'e Stoudemire
8   2014-15  Dallas Mavericks  DAL  NBA     $485,670  Amar'e Sto

### Season Totals

In [7]:
def get_players_totals(tables):
    """Extract Active players Season data for players from all tables and store it in dictionary."""
    
    totals_dict={}
    totals_list=[] 
    ### (Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,
    ###         3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS)
    
    # 1. step: try to extract data for a single table
    for tbl in tables:
        headers = tbl('tr')
        #print headers
        first_header = headers[0]
        #print first_header
        th_s = first_header('th')
        #print th_s    
        title = [val.content for val in th_s ]
        #print title
                
        # 2. step: iterate over all tables, extract headings and actual data and combine data into single dict
        rows = tbl('tr')[1:]
        for row in rows:
            
            tds = row('td')
            
            if len(tds[0]('a')) > 0:
                ### if row content is null , set default string ###
                f_content = (lambda x: "" if len(x) == 0 else x.content)
                ### if row a_herf is null , set default string ###
                f_ahref = (lambda x: x.content if len(x('a')) == 0 else x('a')[0].content)
            
                Season       = f_ahref(tds[0])
                Age          = f_content(tds[1])
                Tm           = f_ahref(tds[2])
                Lg           = f_ahref(tds[3])
                Pos          = f_content(tds[4])
                G            = f_content(tds[5])
                GS           = f_content(tds[6])
                MP           = f_content(tds[7])
                FG           = f_content(tds[8])
                FGA          = f_content(tds[9])
                FG_Per       = f_content(tds[10])
                three_P      = f_content(tds[11])
                three_PA     = f_content(tds[12])
                three_P_Per  = f_content(tds[13])
                two_P        = f_content(tds[14])
                two_PA       = f_content(tds[15])
                two_P_Per    = f_content(tds[16])
                eFG_Per      = f_content(tds[17])
                FT           = f_content(tds[18])
                FTA          = f_content(tds[19])
                FT_Per       = f_content(tds[20])
                ORB          = f_content(tds[21])
                DRB          = f_content(tds[22])
                TRB          = f_content(tds[23])
                AST          = f_content(tds[24])
                STL          = f_content(tds[25])
                BLK          = f_content(tds[26])
                TOV          = f_content(tds[27])
                PF           = f_content(tds[28])
                PTS          = f_content(tds[29])
                Tran         = ""

                totals_list=[Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG_Per,three_P,three_PA,
                             three_P_Per,two_P,two_PA,two_P_Per,eFG_Per,FT,FTA,FT_Per,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS]

                ### if Tm == TOT , current yeat had transactions ###
                if Tm == "TOT":
                    TranFlag = "Y"
                else:
                    TranFlag = "N"
                
                ### append TranFlag to totals_list ###
                totals_list.append(TranFlag)
                
                ### set totals_dict key = Season , value = totals_list ###
                if Season not in totals_dict:
                    totals_dict[Season+Tm]=totals_list

    return totals_dict

### Season Game ###
seasons_totals_dict = get_players_totals(player_tables_by_type[u'totals'])

if len(seasons_totals_dict) > 0 :
    #print seasons_totals_dict
    df_seasons = pd.DataFrame(seasons_totals_dict.values())
    df_seasons['name'] = player_name   
    df_seasons.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per','three_P','three_PA',
                        'three_P_Per','two_P','two_PA','two_P_Per','eFG_Per','FT','FTA','FT_Per','ORB','DRB',
                        'TRB','AST','STL','BLK','TOV','PF','PTS','TranFlag','Name']
    print df_seasons.sort_index(by='Season')

     Season Age   Tm   Lg Pos   G  GS    MP   FG   FGA        ...          \
7   2002-03  20  PHO  NBA  PF  82  71  2570  392   830        ...           
10  2003-04  21  PHO  NBA  PF  55  53  2025  411   865        ...           
14  2004-05  22  PHO  NBA   C  80  80  2889  747  1336        ...           
4   2005-06  23  PHO  NBA   C   3   3    50    9    27        ...           
12  2006-07  24  PHO  NBA   C  82  78  2689  607  1055        ...           
5   2007-08  25  PHO  NBA   C  79  79  2677  714  1211        ...           
11  2008-09  26  PHO  NBA  PF  53  53  1948  404   749        ...           
13  2009-10  27  PHO  NBA  PF  82  82  2838  704  1264        ...           
0   2010-11  28  NYK  NBA   C  78  78  2870  744  1482        ...           
6   2011-12  29  NYK  NBA  PF  47  47  1543  316   654        ...           
2   2012-13  30  NYK  NBA  PF  29   0   682  154   267        ...           
3   2013-14  31  NYK  NBA   C  65  21  1466  311   558        ...           

### Season Per 100 Poss

In [8]:
def get_players_Per_Poss(tables):
    """Extract Active players Per 100 Poss data for players from all tables and store it in dictionary."""
    
    per_poss_dict={}
    per_poss_list=[] 

    # 1. step: try to extract data for a single table
    for tbl in tables:
        headers = tbl('tr')
        #print headers
        first_header = headers[0]
        #print first_header
        th_s = first_header('th')
        #print th_s    
        title = [val.content for val in th_s ]
        #print title
                
        # 2. step: iterate over all tables, extract headings and actual data and combine data into single dict
        rows = tbl('tr')[1:]
        for row in rows:
            
            tds = row('td')
            
            if len(tds[0]('a')) > 0:
                ### if row content is null , set default string ###
                f_content = (lambda x: "" if len(x) == 0 else x.content)
                ### if row a_herf is null , set default string ###
                f_ahref = (lambda x: x.content if len(x('a')) == 0 else x('a')[0].content)
            
                Season      = f_ahref(tds[0])
                Age         = f_content(tds[1])
                Tm          = f_ahref(tds[2])
                Lg          = f_ahref(tds[3])
                Pos         = f_content(tds[4])
                G           = f_content(tds[5])
                GS          = f_content(tds[6])
                MP          = f_content(tds[7])
                FG          = f_content(tds[8])
                FGA         = f_content(tds[9])
                FG_Per      = f_content(tds[10])
                three_P     = f_content(tds[11])
                three_PA    = f_content(tds[12])
                three_P_Per = f_content(tds[13])
                two_P       = f_content(tds[14])
                two_PA      = f_content(tds[15])
                two_P_Per   = f_content(tds[16])
                FT          = f_content(tds[17])
                FTA         = f_content(tds[18])
                FT_Per      = f_content(tds[19])
                ORB         = f_content(tds[20])
                DRB         = f_content(tds[21])
                TRB         = f_content(tds[22])
                AST         = f_content(tds[23])
                STL         = f_content(tds[24])
                BLK         = f_content(tds[25])
                TOV         = f_content(tds[26])
                PF          = f_content(tds[27])
                PTS         = f_content(tds[28])
                undef1      = f_content(tds[29])
                ORtg        = f_content(tds[30])
                DRtg        = f_content(tds[31])

                per_poss_list=[Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG_Per,three_P,three_PA,three_P_Per
                               ,two_P,two_PA,two_P_Per,FT,FTA,FT_Per,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg]
                              
                ### set per_poss_dict key = Season + Tm , value = per_poss_list ###
                if Season not in per_poss_dict:
                    per_poss_dict[Season+Tm]=per_poss_list

    return per_poss_dict

### Season Per 100 Poss ###
Season_Per_Poss_dict = get_players_Per_Poss(player_tables_by_type[u'per_poss'])

if len(Season_Per_Poss_dict) > 0 :
    #print seasons_totals_dict
    df_Season_Per_Poss = pd.DataFrame(Season_Per_Poss_dict.values())
    df_Season_Per_Poss['name'] = player_name   
    df_Season_Per_Poss.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per',
                                'three_P','three_PA','three_P_Per','two_P','two_PA','two_P_Per',
                                'FT','FTA','FT_Per','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','ORtg','DRtg','Name']
    print df_Season_Per_Poss.sort_index(by='Season')

     Season Age   Tm   Lg Pos   G  GS    MP    FG   FGA        ...          \
7   2002-03  20  PHO  NBA  PF  82  71  2570   8.0  16.9        ...           
10  2003-04  21  PHO  NBA  PF  55  53  2025  10.5  22.1        ...           
14  2004-05  22  PHO  NBA   C  80  80  2889  12.9  23.1        ...           
4   2005-06  23  PHO  NBA   C   3   3    50   9.0  27.0        ...           
12  2006-07  24  PHO  NBA   C  82  78  2689  11.3  19.7        ...           
5   2007-08  25  PHO  NBA   C  79  79  2677  13.2  22.5        ...           
11  2008-09  26  PHO  NBA  PF  53  53  1948  10.4  19.2        ...           
13  2009-10  27  PHO  NBA  PF  82  82  2838  12.5  22.4        ...           
0   2010-11  28  NYK  NBA   C  78  78  2870  13.0  25.9        ...           
6   2011-12  29  NYK  NBA  PF  47  47  1543  10.6  21.8        ...           
2   2012-13  30  NYK  NBA  PF  29   0   682  12.1  20.9        ...           
3   2013-14  31  NYK  NBA   C  65  21  1466  11.3  20.2        .

###Merge  df_Season 、df_Per Poss and df_Salary

In [9]:
df_seasons = pd.merge(df_seasons, df_Season_Per_Poss[['Season', 'Tm','ORtg','DRtg']], on=['Season', 'Tm'])
df_seasons = pd.merge(df_seasons, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
print df_seasons.sort_index(by='Season')

     Season Age   Tm   Lg Pos   G  GS    MP   FG   FGA     ...      STL  BLK  \
7   2002-03  20  PHO  NBA  PF  82  71  2570  392   830     ...       62   87   
9   2003-04  21  PHO  NBA  PF  55  53  2025  411   865     ...       64   89   
13  2004-05  22  PHO  NBA   C  80  80  2889  747  1336     ...       77  130   
4   2005-06  23  PHO  NBA   C   3   3    50    9    27     ...        1    3   
11  2006-07  24  PHO  NBA   C  82  78  2689  607  1055     ...       78  110   
5   2007-08  25  PHO  NBA   C  79  79  2677  714  1211     ...       64  163   
10  2008-09  26  PHO  NBA  PF  53  53  1948  404   749     ...       49   57   
12  2009-10  27  PHO  NBA  PF  82  82  2838  704  1264     ...       52   83   
0   2010-11  28  NYK  NBA   C  78  78  2870  744  1482     ...       71  150   
6   2011-12  29  NYK  NBA  PF  47  47  1543  316   654     ...       38   45   
2   2012-13  30  NYK  NBA  PF  29   0   682  154   267     ...       10   21   
3   2013-14  31  NYK  NBA   C  65  21  1

### Playoffs Totals

In [10]:
playoffs_totals_dict = get_players_totals(player_tables_by_type[u'playoffs_totals'])
if len(playoffs_totals_dict) > 0 :
    #print playoffs_totals_dict
    df_playoffs = pd.DataFrame(playoffs_totals_dict.values())
    df_playoffs['name'] = player_name
    df_playoffs.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per','three_P','three_PA',
                        'three_P_Per','two_P','two_PA','two_P_Per','eFG_Per','FT','FTA','FT_Per','ORB','DRB',
                        'TRB','AST','STL','BLK','TOV','PF','PTS','TranFlag','Name']
    print df_playoffs.sort_index(by='Season')

    Season Age   Tm   Lg Pos   G  GS   MP   FG  FGA        ...          DRB  \
4  2002-03  20  PHO  NBA  PF   6   6  203   34   65        ...           31   
8  2004-05  22  PHO  NBA   C  15  15  601  153  284        ...          100   
6  2006-07  24  PHO  NBA   C  10  10  343   91  174        ...           91   
2  2007-08  25  PHO  NBA   C   5   5  204   48   99        ...           30   
7  2009-10  27  PHO  NBA  PF  16  16  584  124  239        ...           60   
0  2010-11  28  NYK  NBA   C   4   4  134   21   55        ...           20   
3  2011-12  29  NYK  NBA  PF   4   4  139   20   36        ...           18   
1  2012-13  30  NYK  NBA  PF   4   0   33    5   13        ...            5   
5  2014-15  32  DAL  NBA  PF   5   0   75   15   35        ...            9   

   TRB AST STL BLK TOV  PF  PTS TranFlag               Name  
4   47   7  10   9  19  28   85        N  Amar'e Stoudemire  
8  160  18  10  30  47  59  449        N  Amar'e Stoudemire  
6  121   6  13  19  19 

###Playoffs Per 100 Poss

In [11]:
Playoffs_Per_Poss_dict = get_players_Per_Poss(player_tables_by_type[u'playoffs_per_poss'])

if len(Playoffs_Per_Poss_dict) > 0 :    
    df_Playoffs_Per_Poss = pd.DataFrame(Playoffs_Per_Poss_dict.values())
    df_Playoffs_Per_Poss['name'] = player_name   
    df_Playoffs_Per_Poss.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per',
                                'three_P','three_PA','three_P_Per','two_P','two_PA','two_P_Per',
                                'FT','FTA','FT_Per','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','ORtg','DRtg','Name']
    print df_Playoffs_Per_Poss.sort_index(by='Season')

    Season Age   Tm   Lg Pos   G  GS   MP    FG   FGA        ...          \
4  2002-03  20  PHO  NBA  PF   6   6  203   8.8  16.8        ...           
8  2004-05  22  PHO  NBA   C  15  15  601  13.0  24.1        ...           
6  2006-07  24  PHO  NBA   C  10  10  343  13.5  25.9        ...           
2  2007-08  25  PHO  NBA   C   5   5  204  12.2  25.2        ...           
7  2009-10  27  PHO  NBA  PF  16  16  584  11.2  21.6        ...           
0  2010-11  28  NYK  NBA   C   4   4  134   8.2  21.5        ...           
3  2011-12  29  NYK  NBA  PF   4   4  139   7.9  14.3        ...           
1  2012-13  30  NYK  NBA  PF   4   0   33   8.4  21.8        ...           
5  2014-15  32  DAL  NBA  PF   5   0   75   9.4  22.0        ...           

    TRB  AST  STL  BLK  TOV    PF   PTS ORtg DRtg               Name  
4  12.2  1.8  2.6  2.3  4.9   7.2  22.0   96   94  Amar'e Stoudemire  
8  13.6  1.5  0.8  2.5  4.0   5.0  38.1  122  112  Amar'e Stoudemire  
6  18.0  0.9  1.9  2.8  2.

###Merge  df_Playoffs 、df_Playoffs Per Poss and df_Salary

In [12]:
df_playoffs = pd.merge(df_playoffs, df_Playoffs_Per_Poss[['Season', 'Tm','ORtg','DRtg']], on=['Season', 'Tm'])
df_playoffs = pd.merge(df_playoffs, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
print df_playoffs.sort_index(by='Season')

    Season Age   Tm   Lg Pos   G  GS   MP   FG  FGA     ...      STL BLK TOV  \
4  2002-03  20  PHO  NBA  PF   6   6  203   34   65     ...       10   9  19   
8  2004-05  22  PHO  NBA   C  15  15  601  153  284     ...       10  30  47   
6  2006-07  24  PHO  NBA   C  10  10  343   91  174     ...       13  19  19   
2  2007-08  25  PHO  NBA   C   5   5  204   48   99     ...        7  12  14   
7  2009-10  27  PHO  NBA  PF  16  16  584  124  239     ...       11  24  40   
0  2010-11  28  NYK  NBA   C   4   4  134   21   55     ...        1   3  12   
3  2011-12  29  NYK  NBA  PF   4   4  139   20   36     ...        5   1  12   
1  2012-13  30  NYK  NBA  PF   4   0   33    5   13     ...        0   0   3   
5  2014-15  32  DAL  NBA  PF   5   0   75   15   35     ...        1   3   7   

   PF  PTS TranFlag               Name ORtg DRtg       Salary  
4  28   85        N  Amar'e Stoudemire   96   94   $1,767,120  
8  59  449        N  Amar'e Stoudemire  122  112   $2,032,200  
6  39  

###Season Advanced 

In [13]:
def get_players_Advance(tables):
    """Extract Active players Advance data for players from all tables and store it in dictionary."""
    
    Advance_dict={}
    Advance_list=[] 

    # 1. step: try to extract data for a single table
    for tbl in tables:
        headers = tbl('tr')
        #print headers
        first_header = headers[0]
        #print first_header
        th_s = first_header('th')
        #print th_s    
        title = [val.content for val in th_s ]
        #print title
                
        # 2. step: iterate over all tables, extract headings and actual data and combine data into single dict
        rows = tbl('tr')[1:]
        for row in rows:
            
            tds = row('td')
            
            if len(tds[0]('a')) > 0:
                ### if row content is null , set default string ###
                f_content = (lambda x: "" if len(x) == 0 else x.content)
                ### if row a_herf is null , set default string ###
                f_ahref = (lambda x: x.content if len(x('a')) == 0 else x('a')[0].content)
            
                Season    = f_ahref(tds[0])
                Age       = f_content(tds[1])
                Tm        = f_ahref(tds[2])
                Lg        = f_ahref(tds[3])
                Pos       = f_content(tds[4 ])
                G         = f_content(tds[5 ])
                MP        = f_content(tds[6 ])
                PER       = f_content(tds[7 ])
                TS_Per    = f_content(tds[8 ])
                three_PAr = f_content(tds[9 ])
                FTr       = f_content(tds[10])
                ORB_Per   = f_content(tds[11])
                DRB_Per   = f_content(tds[12])
                TRB_Per   = f_content(tds[13])
                AST_Per   = f_content(tds[14])
                STL_Per   = f_content(tds[15])
                BLK_Per   = f_content(tds[16])
                TOV_Per   = f_content(tds[17])
                USG_Per   = f_content(tds[18])
                undef1    = f_content(tds[19])
                OWS       = f_content(tds[20])
                DWS       = f_content(tds[21])
                WS        = f_content(tds[22])
                WS48      = f_content(tds[23])
                undef2    = f_content(tds[24])
                OBPM      = f_content(tds[25])
                DBPM      = f_content(tds[26])
                BPM       = f_content(tds[27])
                VORP      = f_content(tds[28])
                Tran      = ""

                Advance_list=[Season,Age,Tm,Lg,Pos,G,MP,PER,TS_Per,three_PAr,FTr,ORB_Per,DRB_Per,TRB_Per,AST_Per,
                              STL_Per,BLK_Per,TOV_Per,USG_Per,OWS,DWS,WS,WS48,OBPM,DBPM,BPM,VORP]
                
                ### if Tm == TOT , current yeat had transactions ###
                if Tm == "TOT":
                    TranFlag = "Y"
                else:
                    TranFlag = "N"
                
                ### append TranFlag to Advance_list ###
                Advance_list.append(TranFlag)
                #print Advance_list
                
                ### set Advance_dict key = Season , value = Advance_list ###
                if Season in Advance_list:
                    Advance_dict[Season+Tm]=Advance_list
                
    return Advance_dict

### Advance ###
seasons_Advance_dict = get_players_Advance(player_tables_by_type[u'advanced'])

if len(seasons_Advance_dict) > 0 :
    #print seasons_totals_dict
    df_Advance = pd.DataFrame(seasons_Advance_dict.values())
    df_Advance['name'] = player_name   
    df_Advance.columns=['Season','Age','Tm','Lg','Pos','G','MP','PER','TS_Per','Three_Per','FT_Per','ORB_Per','DRB_Per','TRB_Per',
                        'AST_Per','STL_Per','BLK_Per','TOV_Per','USG_Per','OWS','DWS','WS','WS48','OBPM','DBPM','BPM','VORP',
                        'TranFlag','Name']
    print df_Advance.sort_index(by='Season')

     Season Age   Tm   Lg Pos   G    MP   PER TS_Per three_PAr  \
7   2002-03  20  PHO  NBA  PF  82  2570  16.2   .530      .012   
10  2003-04  21  PHO  NBA  PF  55  2025  19.8   .536      .006   
14  2004-05  22  PHO  NBA   C  80  2889  26.6   .617      .012   
4   2005-06  23  PHO  NBA   C   3    50  17.8   .420      .037   
12  2006-07  24  PHO  NBA   C  82  2689  23.1   .637      .003   
5   2007-08  25  PHO  NBA   C  79  2677  27.6   .656      .026   
11  2008-09  26  PHO  NBA  PF  53  1948  20.3   .617      .009   
13  2009-10  27  PHO  NBA  PF  82  2838  22.6   .615      .005   
0   2010-11  28  NYK  NBA   C  78  2870  22.7   .565      .016   
6   2011-12  29  NYK  NBA  PF  47  1543  17.7   .541      .032   
2   2012-13  30  NYK  NBA  PF  29   682  22.1   .637      .000   
3   2013-14  31  NYK  NBA   C  65  1466  18.8   .596      .000   
8   2014-15  32  TOT  NBA  PF  59  1245  20.3   .595      .008   
9   2014-15  32  DAL  NBA  PF  23   380  22.3   .605      .006   
1   2014-1

###Merge  Salary and Advance

In [14]:
df_Advance = pd.merge(df_Advance, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
print df_Advance.sort_index(by='Season')

     Season Age   Tm   Lg Pos   G    MP   PER TS_Per three_PAr     ...       \
7   2002-03  20  PHO  NBA  PF  82  2570  16.2   .530      .012     ...        
9   2003-04  21  PHO  NBA  PF  55  2025  19.8   .536      .006     ...        
13  2004-05  22  PHO  NBA   C  80  2889  26.6   .617      .012     ...        
4   2005-06  23  PHO  NBA   C   3    50  17.8   .420      .037     ...        
11  2006-07  24  PHO  NBA   C  82  2689  23.1   .637      .003     ...        
5   2007-08  25  PHO  NBA   C  79  2677  27.6   .656      .026     ...        
10  2008-09  26  PHO  NBA  PF  53  1948  20.3   .617      .009     ...        
12  2009-10  27  PHO  NBA  PF  82  2838  22.6   .615      .005     ...        
0   2010-11  28  NYK  NBA   C  78  2870  22.7   .565      .016     ...        
6   2011-12  29  NYK  NBA  PF  47  1543  17.7   .541      .032     ...        
2   2012-13  30  NYK  NBA  PF  29   682  22.1   .637      .000     ...        
3   2013-14  31  NYK  NBA   C  65  1466  18.8   .596

###Playoffs Advanced

In [15]:
### Advance ###
playoff_Advance_dict = get_players_Advance(player_tables_by_type[u'playoffs_advanced'])

if len(playoff_Advance_dict) > 0 :    
    df_playoff_Advance = pd.DataFrame(playoff_Advance_dict.values())
    df_playoff_Advance['name'] = player_name   
    df_playoff_Advance.columns=['Season','Age','Tm','Lg','Pos','G','MP','PER','TS_Per','Three_Per','FT_Per','ORB_Per','DRB_Per','TRB_Per',
                        'AST_Per','STL_Per','BLK_Per','TOV_Per','USG_Per','OWS','DWS','WS','WS48','OBPM','DBPM','BPM','VORP',
                        'TranFlag','Name']
    print df_playoff_Advance.sort_index(by='Season')

    Season Age   Tm   Lg Pos   G   MP   PER TS_Per three_PAr  \
4  2002-03  20  PHO  NBA  PF   6  203  15.9   .550      .015   
8  2004-05  22  PHO  NBA   C  15  601  27.6   .616      .011   
6  2006-07  24  PHO  NBA   C  10  343  29.6   .591      .017   
2  2007-08  25  PHO  NBA   C   5  204  18.9   .517      .040   
7  2009-10  27  PHO  NBA  PF  16  584  21.2   .589      .017   
0  2010-11  28  NYK  NBA   C   4  134  10.7   .442      .000   
3  2011-12  29  NYK  NBA  PF   4  139  16.9   .631      .000   
1  2012-13  30  NYK  NBA  PF   4   33   9.8   .508      .077   
5  2014-15  32  DAL  NBA  PF   5   75  11.3   .479      .000   

         ...           OWS  DWS    WS    WS48  OBPM  DBPM    BPM  VORP  \
4        ...           0.0  0.5   0.5    .110  -1.0   3.9    2.9   0.2   
8        ...           2.4  0.3   2.7    .218   1.5   0.7    2.2   0.6   
6        ...           1.3  0.7   1.9    .269   1.3   1.3    2.6   0.4   
2        ...           0.0  0.3   0.3    .077  -2.4   1.8   -0.

###Merge Salary and Playoffs Advance

In [16]:
df_playoff_Advance = pd.merge(df_playoff_Advance, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
print df_playoff_Advance.sort_index(by='Season')

    Season Age   Tm   Lg Pos   G   MP   PER TS_Per three_PAr     ...       \
4  2002-03  20  PHO  NBA  PF   6  203  15.9   .550      .015     ...        
8  2004-05  22  PHO  NBA   C  15  601  27.6   .616      .011     ...        
6  2006-07  24  PHO  NBA   C  10  343  29.6   .591      .017     ...        
2  2007-08  25  PHO  NBA   C   5  204  18.9   .517      .040     ...        
7  2009-10  27  PHO  NBA  PF  16  584  21.2   .589      .017     ...        
0  2010-11  28  NYK  NBA   C   4  134  10.7   .442      .000     ...        
3  2011-12  29  NYK  NBA  PF   4  139  16.9   .631      .000     ...        
1  2012-13  30  NYK  NBA  PF   4   33   9.8   .508      .077     ...        
5  2014-15  32  DAL  NBA  PF   5   75  11.3   .479      .000     ...        

   DWS    WS    WS48  OBPM  DBPM    BPM  VORP TranFlag               Name  \
4  0.5   0.5    .110  -1.0   3.9    2.9   0.2        N  Amar'e Stoudemire   
8  0.3   2.7    .218   1.5   0.7    2.2   0.6        N  Amar'e Stoudemire  

### Output DataFrame to CSV

In [17]:
def Output_CSV(output_df,location):
    #output_file = open("e:\\nba\\Playoffs_Totals.csv", "ab")
    output_file = open(location, "ab")
    output_df.to_csv(output_file,index=False)
    output_file.close()

#location = "e:\\nba\\Salary_extraction.csv"
#Output_CSV(df_salaries,location)

###Player information

In [18]:
def All_player(player_name,player_information_url):
    ### Information table ###
    (player_information_tables,player_tables_by_type) = player_information(player_information_url)
    ###########################################################################################################################
    ### Salary ###
    salaries_dict=get_players_salaries(player_tables_by_type[u'salaries'])
    
    if len(salaries_dict) > 0:
        df_salaries = pd.DataFrame(salaries_dict.values())
        df_salaries['name'] = player_name
        df_salaries.columns=['Season','Team','Tm','Lg','Salary','Name']
        salary_location = "e:\\nba\\Salary_extraction.csv"
        Output_CSV(df_salaries.sort_index(by='Season'),salary_location)
    ###########################################################################################################################
    ### Season Game Totals ###
    seasons_totals_dict = get_players_totals(player_tables_by_type[u'totals'])
    
    if (len(seasons_totals_dict) > 0 ) and (len(salaries_dict) > 0):    
        df_seasons = pd.DataFrame(seasons_totals_dict.values())
        df_seasons['name'] = player_name
        df_seasons.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per','three_P','three_PA',
                            'three_P_Per','two_P','two_PA','two_P_Per','eFG_Per','FT','FTA','FT_Per','ORB','DRB',
                            'TRB','AST','STL','BLK','TOV','PF','PTS','TranFlag','Name']
    
    ### Season Per 100 Poss ###
    Season_Per_Poss_dict = get_players_Per_Poss(player_tables_by_type[u'per_poss'])

    if len(Season_Per_Poss_dict) > 0 :
        #print seasons_totals_dict
        df_Season_Per_Poss = pd.DataFrame(Season_Per_Poss_dict.values())
        df_Season_Per_Poss['name'] = player_name   
        df_Season_Per_Poss.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per','three_P','three_PA',
                                    'three_P_Per','two_P','two_PA','two_P_Per','FT','FTA','FT_Per','ORB','DRB',
                                    'TRB','AST','STL','BLK','TOV','PF','PTS','ORtg','DRtg','Name']
    
    ### Mergre Season Total 、Per 100 Poss['ORtg','DRtg'] and Salary ###
    if (len(seasons_totals_dict) > 0 ) and (len(salaries_dict) > 0):
        df_seasons = pd.merge(df_seasons, df_Season_Per_Poss[['Season', 'Tm','ORtg','DRtg']], on=['Season', 'Tm'])
        df_seasons = pd.merge(df_seasons, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
    
        ### Output DataFrame to CSV
        Season_location = "e:\\nba\\Season_Totals.csv"
        Output_CSV(df_seasons.sort_index(by='Season'),Season_location)
    
    ###########################################################################################################################
    ### Playoffs Game Totals ###
    playoffs_totals_dict = get_players_totals(player_tables_by_type[u'playoffs_totals'])
    if len(playoffs_totals_dict) > 0 :
        #print playoffs_totals_dict
        df_playoffs = pd.DataFrame(playoffs_totals_dict.values())
        df_playoffs['name'] = player_name
        df_playoffs.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per','three_P','three_PA',
                            'three_P_Per','two_P','two_PA','two_P_Per','eFG_Per','FT','FTA','FT_Per','ORB','DRB',
                            'TRB','AST','STL','BLK','TOV','PF','PTS','TranFlag','Name']
        
    ### Playoffs Per 100 Poss ###
    Playoffs_Per_Poss_dict = get_players_Per_Poss(player_tables_by_type[u'playoffs_per_poss'])

    if len(Playoffs_Per_Poss_dict) > 0 :    
        df_Playoffs_Per_Poss = pd.DataFrame(Playoffs_Per_Poss_dict.values())
        df_Playoffs_Per_Poss['name'] = player_name   
        df_Playoffs_Per_Poss.columns=['Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG_Per',
                                    'three_P','three_PA','three_P_Per','two_P','two_PA','two_P_Per',
                                    'FT','FTA','FT_Per','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','ORtg','DRtg','Name']
    
    ### Mergre Season Total 、Per 100 Poss['ORtg','DRtg'] and Salary ###
    if (len(playoffs_totals_dict) > 0 ) and (len(salaries_dict) > 0):
        df_playoffs = pd.merge(df_playoffs, df_Playoffs_Per_Poss[['Season', 'Tm','ORtg','DRtg']], on=['Season', 'Tm'])
        df_playoffs = pd.merge(df_playoffs, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
    
        ### Output DataFrame to CSV
        Playoffs_location = "e:\\nba\\Playoffs_Totals.csv"
        Output_CSV(df_playoffs.sort_index(by='Season'),Playoffs_location)

    ###########################################################################################################################
    ### Advance Game Totals ###
    seasons_Advance_dict  = get_players_Advance(player_tables_by_type[u'advanced'])
    
    if (len(seasons_Advance_dict) > 0 ) and (len(salaries_dict) > 0):
        df_Advance = pd.DataFrame(seasons_Advance_dict.values())
        df_Advance['name'] = player_name   
        df_Advance.columns=['Season','Age','Tm','Lg','Pos','G','MP','PER','TS_Per','three_Per','FT_Per','ORB_Per','DRB_Per','TRB_Per',
                        'AST_Per','STL_Per','BLK_Per','TOV_Per','USG_Per','OWS','DWS','WS','WS48','OBPM','DBPM','BPM','VORP',
                        'TranFlag','Name']
        df_Advance = pd.merge(df_Advance, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
        Season_Advance_location = "e:\\nba\\Season_Advance_Totals.csv"
        Output_CSV(df_Advance.sort_index(by='Season'),Season_Advance_location)        
    
    ### Playoffs Advance Game Totals ###
    playoff_Advance_dict = get_players_Advance(player_tables_by_type[u'playoffs_advanced'])
    
    if (len(playoff_Advance_dict) > 0 ) and (len(salaries_dict) > 0):    
        df_playoff_Advance = pd.DataFrame(playoff_Advance_dict.values())
        df_playoff_Advance['name'] = player_name
        df_playoff_Advance.columns=['Season','Age','Tm','Lg','Pos','G','MP','PER','TS_Per','three_Per','FT_Per','ORB_Per','DRB_Per','TRB_Per',
                        'AST_Per','STL_Per','BLK_Per','TOV_Per','USG_Per','OWS','DWS','WS','WS48','OBPM','DBPM','BPM','VORP',
                        'TranFlag','Name']
        df_playoff_Advance = pd.merge(df_playoff_Advance, df_salaries[['Season', 'Tm','Salary']], on=['Season', 'Tm'])
        Playoffs_Advance_location = "e:\\nba\\Playoffs_Advance_Totals.csv"
        Output_CSV(df_playoff_Advance.sort_index(by='Season'),Playoffs_Advance_location)    

    ###########################################################################################################################

### Name & Information URL ###
player_list = df_player['Player'].values.tolist()
#print len(player_list)

player_link_list = df_player['link'].values.tolist()
#print len(player_link_list)

for player_name,player_information_url in zip(player_list, player_link_list):    
    All_player(player_name,player_information_url)
